In [1]:
# helper functions
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from average_precision import apk

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)

In [2]:
%%time
import pandas as pd

transactions = pd.read_csv('data/transactions_train.csv', dtype={"article_id": "str"})
customers = pd.read_csv('data/customers.csv')
articles = pd.read_csv('data/articles.csv', dtype={"article_id": "str"})

CPU times: total: 33.1 s
Wall time: 39.1 s


In [3]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,0685687004,0.016932,2


In [6]:
transactions.customer_id[0]

'000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318'

In [7]:
len(transactions.customer_id[0])

64

In [4]:
transactions.memory_usage(deep=True)

Index                      128
t_dat               2129817708
customer_id         3846387204
article_id          2129817708
price                254306592
sales_channel_id     254306592
dtype: int64

In [5]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 8.0 GB


In [8]:
%%time
transactions['customer_id'].nunique()

CPU times: total: 6.11 s
Wall time: 7.08 s


1362281

In [9]:
%%time
transactions['customer_id'] = customer_hex_id_to_int(transactions['customer_id'])
transactions['customer_id'].nunique()

CPU times: total: 21.4 s
Wall time: 28.1 s


1362281

In [10]:
transactions.memory_usage(deep=True)

Index                      128
t_dat               2129817708
customer_id          254306592
article_id          2129817708
price                254306592
sales_channel_id     254306592
dtype: int64

In [11]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       uint64 
 2   article_id        object 
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(1), object(2), uint64(1)
memory usage: 4.7 GB


In [12]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,18439897732908966680,0663713001,0.050831,2
1,2018-09-20,18439897732908966680,0541518023,0.030492,2
2,2018-09-20,10112112306570742978,0505221004,0.015237,2
3,2018-09-20,10112112306570742978,0685687003,0.016932,2
4,2018-09-20,10112112306570742978,0685687004,0.016932,2


In [13]:
%%time

transactions.t_dat = pd.to_datetime(transactions.t_dat, format='%Y-%m-%d')

CPU times: total: 2.81 s
Wall time: 3.33 s


In [14]:
transactions['week'] = 104 - (transactions.t_dat.max() - transactions.t_dat).dt.days // 7

In [15]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       uint64        
 2   article_id        object        
 3   price             float64       
 4   sales_channel_id  int64         
 5   week              int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1), uint64(1)
memory usage: 3.2 GB


In [16]:
transactions.article_id = article_id_str_to_int(transactions.article_id)
articles.article_id = article_id_str_to_int(articles.article_id)

transactions.week = transactions.week.astype('int8')
transactions.sales_channel_id = transactions.sales_channel_id.astype('int8')
transactions.price = transactions.price.astype('float32')

In [33]:
transactions.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 6 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       uint64        
 2   article_id        int32         
 3   price             float32       
 4   sales_channel_id  int8          
 5   week              int8          
dtypes: datetime64[ns](1), float32(1), int32(1), int8(2), uint64(1)
memory usage: 788.2 MB


In [30]:
transactions.drop(columns='t_dat').info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   customer_id       uint64 
 1   article_id        int32  
 2   price             float32
 3   sales_channel_id  int8   
 4   week              int8   
dtypes: float32(1), int32(1), int8(2), uint64(1)
memory usage: 545.7 MB


In [34]:
customers.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355969 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 512.3 MB


In [35]:
articles.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int32 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [36]:
customers['club_member_status'].unique()

array(['ACTIVE', nan, 'PRE-CREATE', 'LEFT CLUB'], dtype=object)

In [37]:
customers.customer_id = customer_hex_id_to_int(customers.customer_id)
for col in ['FN', 'Active', 'age']:
    customers[col].fillna(-1, inplace=True)
    customers[col] = customers[col].astype('int8')

C:\Users\Nirajan\AppData\Local\Temp\ipykernel_22540\2451866874.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customers[col].fillna(-1, inplace=True)
C:\Users\Nirajan\AppData\Local\Temp\ipykernel_22540\2451866874.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example,

In [38]:
customers.club_member_status = Categorize().fit_transform(customers[['club_member_status']]).club_member_status
customers.postal_code = Categorize().fit_transform(customers[['postal_code']]).postal_code
customers.fashion_news_frequency = Categorize().fit_transform(customers[['fashion_news_frequency']]).fashion_news_frequency

In [39]:
customers.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   customer_id             1371980 non-null  uint64
 1   FN                      1371980 non-null  int8  
 2   Active                  1371980 non-null  int8  
 3   club_member_status      1371980 non-null  int8  
 4   fashion_news_frequency  1371980 non-null  int8  
 5   age                     1371980 non-null  int8  
 6   postal_code             1371980 non-null  int32 
dtypes: int32(1), int8(5), uint64(1)
memory usage: 22.2 MB


In [40]:
for col in articles.columns:
    if articles[col].dtype == 'object':
        articles[col] = Categorize().fit_transform(articles[[col]])[col]

In [41]:
articles.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype
---  ------                        --------------   -----
 0   article_id                    105542 non-null  int32
 1   product_code                  105542 non-null  int64
 2   prod_name                     105542 non-null  int32
 3   product_type_no               105542 non-null  int64
 4   product_type_name             105542 non-null  int16
 5   product_group_name            105542 non-null  int8 
 6   graphical_appearance_no       105542 non-null  int64
 7   graphical_appearance_name     105542 non-null  int8 
 8   colour_group_code             105542 non-null  int64
 9   colour_group_name             105542 non-null  int8 
 10  perceived_colour_value_id     105542 non-null  int64
 11  perceived_colour_value_name   105542 non-null  int8 
 12  perceived_colour_master_id    105542 non-null  int64
 13  perceived_colo

In [42]:
for col in articles.columns:
    if articles[col].dtype == 'int64':
        articles[col] = articles[col].astype('int32')

In [43]:
transactions.sort_values(['t_dat', 'customer_id'], inplace=True)

In [45]:
%%time

transactions.to_parquet('data/transactions_train.parquet')
customers.to_parquet('data/customers.parquet')
articles.to_parquet('data/articles.parquet')

CPU times: total: 5.73 s
Wall time: 6.86 s


In [46]:
%%time
# let's create a 5% sample of the entiriety of the data to speed up dev

sample = 0.05
customers_sample = customers.sample(frac=sample, replace=False)
customers_sample_ids = set(customers_sample['customer_id'])
transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
articles_sample_ids = set(transactions_sample["article_id"])
articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]

customers_sample.to_parquet(f'data/customers_sample_{sample}.parquet', index=False)
transactions_sample.to_parquet(f'data/transactions_train_sample_{sample}.parquet', index=False)
articles_sample.to_parquet(f'data/articles_train_sample_{sample}.parquet', index=False)

CPU times: total: 3.48 s
Wall time: 3.99 s


### Evaluation

In [47]:
from collections import defaultdict

val_week_purchases_by_cust = defaultdict(list)

val_week_purchases_by_cust.update(
    transactions[transactions.week == transactions.week.max()] \
        .groupby('customer_id')['article_id'] \
        .apply(list) \
        .to_dict()
)

In [50]:
pd.to_pickle(dict(val_week_purchases_by_cust), 'data/val_week_purchases_by_cust.pkl')

In [81]:
sample_sub = pd.read_csv('data/sample_submission.csv')
valid_gt = customer_hex_id_to_int(sample_sub.customer_id) \
    .map(val_week_purchases_by_cust) \
    .apply(lambda xx: ' '.join('0' + str(x) for x in xx))

sample_sub.prediction = valid_gt
sample_sub.to_parquet('data/validation_ground_truth.parquet', index=False)

In [82]:
def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

In [83]:
def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

### Strong starting point

In [84]:
%%time
final_week = transactions[transactions.week == transactions.week.max()]

CPU times: total: 62.5 ms
Wall time: 52.9 ms


In [85]:
final_week['customer_id'].value_counts().describe()

count    68984.000000
mean         3.483576
std          3.535144
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max        104.000000
Name: count, dtype: float64

In [86]:
bestsellers_last_week = set(transactions[transactions.week == transactions.week.max()].article_id.value_counts().index[:12])
bestsellers_week_ago = set(transactions[transactions.week == transactions.week.max()-1].article_id.value_counts().index[:12])

In [87]:
bestsellers_last_week

{448509014,
 714790020,
 751471001,
 762846027,
 866731001,
 909370001,
 915529003,
 915529005,
 918522001,
 923758001,
 924243001,
 924243002}

In [88]:
bestsellers_week_ago

{448509014,
 673677002,
 751471001,
 762846027,
 809238001,
 809238005,
 865799006,
 909370001,
 918292001,
 918522001,
 923758001,
 924243001}

In [89]:
final_week.article_id.isin(bestsellers_last_week).mean(), final_week.article_id.isin(bestsellers_week_ago).mean()

(0.027967924897320556, 0.02349871624686344)

In [90]:
without_final_week = transactions[transactions.week != transactions.week.max()]

In [92]:
%%time
unique_bought_items = 0
last_purchase_repeated = 0
purchases_cust_with_no_history = 0
items_purchased_by_custs_with_no_history = []
week_of_earlier_purchase = []
i = 0

for c_id, df in final_week.groupby('customer_id'):
    purchases_final_week = set(df.article_id)
    unique_bought_items += len(purchases_final_week)
    
    purchase_history = without_final_week[without_final_week.customer_id == c_id]
    purchases_before = set(purchase_history[purchase_history.week == purchase_history.week.max()].article_id)
    week_of_earlier_purchase.append(purchase_history.week.max())
    
    if len(purchases_before) == 0:
        purchases_cust_with_no_history += len(purchases_final_week)
        items_purchased_by_custs_with_no_history += list(purchases_final_week)
    else:
        last_purchase_repeated += len(purchases_final_week.intersection(purchases_before))
    i += 1
    if i == 1000: break

CPU times: total: 33.2 s
Wall time: 37.1 s


In [93]:
wks_since_purchase = []
for week in week_of_earlier_purchase:
    if type(week) == np.int8:
        wks_since_purchase.append(104 - week)

In [98]:
# weeks elapsed between the purchase in the final week and earlier purchase
(pd.value_counts(wks_since_purchase)/len(wks_since_purchase)).head(20).cumsum()

C:\Users\Nirajan\AppData\Local\Temp\ipykernel_22540\2239400022.py:2: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  (pd.value_counts(wks_since_purchase)/len(wks_since_purchase)).head(20).cumsum()
C:\Users\Nirajan\AppData\Local\Temp\ipykernel_22540\2239400022.py:2: FutureWarning: value_counts with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  (pd.value_counts(wks_since_purchase)/len(wks_since_purchase)).head(20).cumsum()


1     0.196312
2     0.305857
3     0.396963
4     0.475054
5     0.539046
7     0.591106
6     0.634490
8     0.676790
13    0.708243
10    0.738612
11    0.767896
12    0.790672
9     0.812364
16    0.827549
15    0.838395
23    0.849241
14    0.860087
18    0.870933
21    0.880694
17    0.889371
Name: count, dtype: float64

In [99]:
unique_bought_items, last_purchase_repeated, purchases_cust_with_no_history

(3039, 80, 238)

In [100]:
np.mean([itm in bestsellers_last_week for itm in items_purchased_by_custs_with_no_history])

0.02100840336134454

In [101]:
%%time
last_three_weeks = without_final_week[without_final_week.week > without_final_week.week.max()-3]

CPU times: total: 62.5 ms
Wall time: 88.3 ms


In [102]:
# this is a slightly different logic to what's in the reference Kaggle kernel
best_sellers = last_three_weeks.groupby('week').apply(lambda df: df.value_counts('article_id').index[:12].tolist())

C:\Users\Nirajan\AppData\Local\Temp\ipykernel_22540\120240434.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_sellers = last_three_weeks.groupby('week').apply(lambda df: df.value_counts('article_id').index[:12].tolist())


In [103]:
best_sellers

week
101    [898694001, 933706001, 751471001, 915526001, 9...
102    [915526001, 751471043, 751471001, 706016001, 9...
103    [909370001, 865799006, 918522001, 924243001, 4...
dtype: object

In [104]:
def purchase_history_to_preds(df):
    week_of_last_purchase = df.week.max()
    last_purchased_basket = df[df.week == week_of_last_purchase]
    purchased_items = last_purchased_basket.value_counts('article_id').index.tolist()
    purchased_items += best_sellers[last_purchased_basket.week.head(1).item()]
    return purchased_items[:12]

In [115]:
%%time

cust2preds = last_three_weeks.groupby(['customer_id']).apply(purchase_history_to_preds)

CPU times: total: 1min 6s
Wall time: 6min 50s


<timed exec>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


### Generate the submission

In [113]:
last_week = last_three_weeks.week.max()
def get_preds_for_customer_id(c_id):
    if c_id in c_ids_with_predictions:
        pred_art_ids = cust2preds[c_id]
    else:
        pred_art_ids = best_sellers[last_week]
    return  ['0' + str(art_id) for art_id in pred_art_ids]

In [116]:
%%time

c_ids_with_predictions = set(cust2preds.keys())
preds = customer_hex_id_to_int(sample_sub.customer_id).map(get_preds_for_customer_id)

CPU times: total: 4.02 s
Wall time: 19.9 s


In [117]:
sample_sub.prediction = preds
sample_sub.prediction = sample_sub.prediction.str.join(' ')

In [118]:
sub_name = 'bestsellers_single_week_logic'

In [121]:
%%time
sample_sub.to_csv(f'data/subs/{sub_name}.csv.gz', index=False)

CPU times: total: 7.81 s
Wall time: 31.4 s


In [122]:
eval_sub(f'data/subs/{sub_name}.csv.gz', skip_cust_with_no_purchases=True)

<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow.lib.Tensor size changed, may indicate binary incompatibility. Expected 64 from C header, got 80 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow.lib.NativeFile size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow.lib.BufferedInputStream size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow.lib.BufferedOutputStream size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow.lib.CompressedInputStream size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
<frozen importlib._bootstrap>:228: RuntimeWarning: pyarrow.lib.CompressedOutputStream size changed, may i

0.022659703348912746